In [2]:
HOME_DIR = "/home_remote"
HOME = "/home/thi.tra.my.nguyen"

import pandas as pd
import glob
import re
import nltk
from nltk.tokenize import RegexpTokenizer
import zipfile
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Dense, MaxPooling1D, Flatten, Dropout, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras import backend as K
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import fasttext
import fasttext.util


import ast
import os

### Word Embedding

Using Wiki pretrained model

In [5]:
ft = fasttext.load_model('/home_remote/fastText/cc.en.300.bin')

In [6]:

# Function to get word embeddings for a document
def get_document_embeddings(document, max_length=100):
    # Tokenize the document and get the first max_length word vectors
    tokens = document.split()[:max_length]
    
    # Apply zero-padding if the document is shorter than max_length
    if len(tokens) < max_length:
        padding_count = max_length - len(tokens)
        tokens.extend(['<PAD>'] * padding_count)

    # Get word embeddings for each token
    embeddings = [ft.get_word_vector(token) for token in tokens]

    return np.array(embeddings)


### Convolutional Neural Network

In [7]:
positives_df_path = os.path.join(HOME_DIR, "positive_df.pkl")
negatives_df_path = os.path.join(HOME_DIR, "negative_df.pkl")

positives = pd.read_pickle(positives_df_path)
negatives = pd.read_pickle(negatives_df_path)

positives['Label'] = 1
negatives['Label'] = 0

#concatenate the two dataframes
df = pd.concat([positives, negatives], ignore_index=True)

In [8]:
subject = df.TrainSubjectId.unique()
#split the subjects into train and test
train_subject, test_subject = train_test_split(subject, test_size=0.2, random_state=42, shuffle=True)
df_train = df[df.TrainSubjectId.isin(train_subject)]
df_test = df[df.TrainSubjectId.isin(test_subject)]
df_train = df_train.sample(frac=1, random_state=42)
df_test = df_test.sample(frac=1, random_state=42)
y_train = df_train.Label
y_test = df_test.Label
X_train = df_train['Text']
X_test = df_test['Text']

In [3]:
# Custom CReLU activation function
def crelu(x):
    pos = K.relu(x)
    neg = K.relu(-x)
    return K.concatenate([pos, neg], axis=-1)

In [3]:
# Word embedding dimensionality
word_vector_dimensions = 300

# Model
model = Sequential()

# Convolutional Layer
model.add(Conv1D(filters=100, kernel_size=2, input_shape=(100, 300)))
model.add(Activation(crelu))
model.add(Activation(lambda x: x * -1))  # Negated activation for CReLU

# 1-Max Pooling Layer
model.add(MaxPooling1D(pool_size=99))

# Fully Connected Layer
model.add(Flatten())
model.add(Dense(200))

# Dropout Layer
model.add(Dropout(0.4))

# 3 Fully Connected Layers
model.add(Dense(100, activation=crelu))
model.add(Dense(50, activation=crelu))

# Final Layer
model.add(Dense(2, activation='softmax'))

# Display the model summary
model.summary()

model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=1e-4), metrics=['f1'])


: 

: 

In [ ]:

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Get the 98th percentile of the predictions
percentile_98 = np.percentile(y_pred, 98)

print("98th Percentile of Predictions:", percentile_98)
